#**Montar directorio de Google drive** 

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Imports**

In [4]:
import tensorflow as tf
#Imports para csv_utils
import os
import pandas as pd
import re
import time
#Imports para TF
import numpy as np
import matplotlib.pyplot as plt
import pickle
%matplotlib inline
from IPython.display import clear_output 
import os
import time
os.chdir('/content/drive/MyDrive/Colab Notebooks/git-repos/ssh/tt-ia_models/')

# **Definiendo Paths**

In [36]:
os.chdir('/content/drive/MyDrive/Colab Notebooks/git-repos/ssh/tt-ia_models/')
root_path = os.getcwd() #Path del directorio root
dataset_path = root_path + '/src/datasets/wheather-dataset/'
models_path = root_path + '/src_colab/Models/'
print(f'root_path: {root_path}')
print(f'dataset_path: {dataset_path}')
print(f'models_path: {models_path}')

root_path: /content/drive/MyDrive/Colab Notebooks/git-repos/ssh/tt-ia_models
dataset_path: /content/drive/MyDrive/Colab Notebooks/git-repos/ssh/tt-ia_models/src/datasets/wheather-dataset/
models_path: /content/drive/MyDrive/Colab Notebooks/git-repos/ssh/tt-ia_models/src_colab/Models/


# **CSV && Plot utils**

In [37]:
def to_csv(list_content,filename,columns):
    COLUMNAS = columns
    town_df = pd.DataFrame(list_content,columns=COLUMNAS)
    town_df.to_csv('csv_dataset/' + filename + '.csv')

def get_csv_data(filename):
    #Lee el .csv
    df = pd.read_csv(filename)
    df.pop('Unnamed: 0')
    return df

def graph_data(dataFrame, param):
    dataFrame.plot(x='FECHA', y=param)
    plt.show()

def get_file_name(fullname):
    parts = fullname.split('.')
    return parts[0]


In [ ]:
#Sección exclusiva para probar csv utils

# **Obteniendo los nombres de los archivos csv a leer**

In [7]:
with open(root_path + '/src_colab/csv_towns_names.pickle', "rb") as a_file:
  FILES = pickle.load(a_file)
  print(FILES)
  print(len(FILES))
    

['14002-ACATLAN DE JUAREZ.csv', '14006-TEOCALTICHE.csv', '14009-AMECA.csv', '14011-LA HUERTA.csv', '14016-IXTLAHUACAN DE LOS MEMBRILLOS.csv', '14017-ATOTONILCO EL ALTO.csv', '14018-ATOYAC.csv', '14023-BOLAÑOS.csv', '14024-TOMATLAN.csv', '14026-COLOTLAN.csv', '14028-CIHUATLAN.csv', '14029-CONCEPCION DE BUENOS AIRES.csv', '14030-ZAPOTLAN EL GRANDE.csv', '14033-LAGOS DE MORENO.csv', '14034-TAMAZULA DE GORDIANO.csv', '14035-MASCOTA.csv', '14036-CUAUTITLAN DE GARCIA BARRAGAN.csv', '14037-HUEJUCAR.csv', '14038-SAN CRISTOBAL DE LA BARRANCA.csv', '14039-CUQUIO.csv', '14040-CHAPALA.csv', '14044-TALPA DE ALLENDE.csv', '14046-AUTLAN DE NAVARRO.csv', '14047-OCOTLAN.csv', '14052-TAPALPA.csv', '14053-HUEJUQUILLA EL ALTO.csv', '14056-SAN MARTIN HIDALGO.csv', '14059-CABO CORRIENTES.csv', '14060-ARANDAS.csv', '14063-ETZATLAN.csv', '14065-GUADALAJARA.csv', '14068-HOSTOTIPAQUILLO.csv', '14070-DEGOLLADO.csv', '14075-JAMAY.csv', '14076-JESUS MARIA.csv', '14078-JUCHITLAN.csv', '14080-YAHUALICA DE GONZALEZ G

# **Windowed dataset**

In [46]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
 series = tf.expand_dims(series, axis=-1)
 #print(series)
 #print(series.shape)
 ds = tf.data.Dataset.from_tensor_slices(series)
 ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
 ds = ds.flat_map(lambda w: w.batch(window_size + 1))
 ds = ds.shuffle(shuffle_buffer)
 ds = ds.map(lambda w: (w[:-1], w[1:]))
 return ds.batch(batch_size).prefetch(1)

# **Definiendo el modelo**

In [39]:
def Neural_network_model():
  tf.keras.backend.clear_session()
  model = tf.keras.models.Sequential([
  tf.keras.layers.Conv1D(filters=60, kernel_size=5,
  strides=1, padding="causal",
  activation="relu",
  input_shape=[None, 1]),
  tf.keras.layers.LSTM(60, return_sequences=True),
  tf.keras.layers.LSTM(60, return_sequences=True),
  tf.keras.layers.Dense(30, activation="relu"),
  tf.keras.layers.Dense(10, activation="relu"),
  tf.keras.layers.Dense(1),
  tf.keras.layers.Lambda(lambda x: x * 400)
  ])
  #model.summary()
  optimizer = tf.keras.optimizers.SGD(lr=1e-6, momentum=0.9)
  model.compile(loss=tf.keras.losses.Huber(), optimizer=optimizer, metrics=["mae"])
  return model

In [40]:
def model_forecast(model, series, window_size):
 ds = tf.data.Dataset.from_tensor_slices(series)
 ds = ds.window(window_size, shift=1, drop_remainder=True)
 ds = ds.flat_map(lambda w: w.batch(window_size))
 ds = ds.batch(32).prefetch(1)
 forecast = model.predict(ds)
 return forecast

In [ ]:
rnn_forecast = model_forecast(model, df['TMAX(°C)'].to_numpy()[..., np.newaxis], window_size)
rnn_forecast = rnn_forecast[len(train) - window_size:-1, -1, 0]
plt.figure(figsize=(10, 6))
#plt.plot(df.iloc[len(train):]['FECHA'], test, color='red')
#plt.plot(df.iloc[len(train):]['FECHA'], rnn_forecast)
plt.plot(date_time[len(train):], test, color='red')
plt.plot(date_time[len(train):], rnn_forecast)
plt.show()

# **Importando el modelo**

In [ ]:
new_model = tf.keras.models.load_model(f'{models_path}{name_model}.h5')

rnn_forecast = model_forecast(new_model, df['TMAX(°C)'].to_numpy()[..., np.newaxis], window_size)
rnn_forecast = rnn_forecast[len(train) - window_size:-1, -1, 0]
plt.figure(figsize=(10, 6))
#plt.plot(df.iloc[len(train):]['FECHA'], test, color='red')
#plt.plot(df.iloc[len(train):]['FECHA'], rnn_forecast)
plt.plot(date_time[len(train):], test, color='red')
plt.plot(date_time[len(train):], rnn_forecast)
plt.show()

# **Entrenamiento de 4 modelos por archivo**

In [41]:
HISTORY = {}
def save_history(filename, var, data):
  if HISTORY.get(filename)==None:
    HISTORY[filename] = {}
  HISTORY[filename][var] = data
  with open(f'{root_path}/src_colab/history.pickle', "wb") as a_file:
    pickle.dump(HISTORY, a_file, protocol=pickle.HIGHEST_PROTOCOL)

def save_last_key(key):
  with open(f'{root_path}/src_colab/key.pickle', "wb") as a_file:
    pickle.dump(key, a_file, protocol=pickle.HIGHEST_PROTOCOL)
    a_file.close()

def get_dump(filename):
  with open(f'{root_path}/src_colab/{filename}', "rb") as a_file:
    output = pickle.load(a_file)
    #print(output)
    return output

def bitacora(info):
  with open(f'{root_path}/src_colab/ModelosCompletados.txt','a+', encoding = "utf-8") as file:
    file.write(info+'\n')

In [ ]:
h = {}
v1 = 'hola'
v2 = 'var2'
if h.get(v1)==None:
  h[v1] = {}
h[v1][v2] = [1,2,3,4,5,6,6]
print(h)
print(h[v1][v2])
if FILES[9] not in NULL_TOWNS:
  print('no esta')
else:
  print('si esta')

{'hola': {'var2': [1, 2, 3, 4, 5, 6, 6]}}
[1, 2, 3, 4, 5, 6, 6]
si esta


In [42]:
#save_last_key([8, 2])
VARIABLES = ['PRECIP(mm)', 'TMAX(°C)', 'TMIN(°C)']
DICT_NAMES = {'PRECIP(mm)':'precip', 'TMAX(°C)':'tmax', 'TMIN(°C)':'tmin'}
NULL_TOWNS = ['14026-COLOTLAN.csv', '14037-HUEJUCAR.csv', '15027-SANTA MARIA DE LOS ANGELES.csv', '14324-TOTATICHE.csv']
HISTORY = get_dump('history.pickle')
print(HISTORY)
print(HISTORY.keys())
print(FILES[9])

{'14002-ACATLAN DE JUAREZ.csv': {'precip': {'loss': [9.412681579589844, 6.334582805633545, 5.313165664672852, 4.836631774902344, 4.476109504699707, 4.257781982421875, 4.065696716308594, 3.901013135910034, 3.7563772201538086, 3.628579616546631, 3.519181251525879, 3.426830768585205, 3.3484842777252197, 3.27925705909729, 3.2152817249298096, 3.1581645011901855, 3.104590654373169, 3.0535714626312256, 3.0028727054595947, 2.9532721042633057, 2.9008121490478516, 2.8467726707458496, 2.793360948562622, 2.7390966415405273, 2.685750722885132, 2.6355905532836914, 2.5919651985168457, 2.5574820041656494, 2.530329942703247, 2.5071940422058105, 2.486821413040161, 2.4703454971313477, 2.4565536975860596, 2.445281505584717, 2.435236692428589, 2.426337480545044, 2.4182615280151367, 2.410738706588745, 2.403637409210205, 2.397420644760132, 2.39164400100708, 2.386058807373047, 2.3807337284088135, 2.3756041526794434, 2.3706302642822266, 2.365891456604004, 2.3613951206207275, 2.3573949337005615, 2.3535702228546

In [11]:
#print(FILES[9])
print(FILES[83])

14355-LA BARCA.csv


In [ ]:
print(get_dump('key.pickle')) #[59, 2]

[75, 0]


In [ ]:
print(get_dump('key.pickle')) #[59, 2]

In [ ]:
tf.random.set_seed(51)
np.random.seed(51)
shuffle_buffer_size = 1000
window_size = 64
batch_size = 365
init_values = get_dump('key.pickle')
key = init_values[0]
LIMIT = 84
#Obtiene el indice de la variable a calcular
index_init = init_values[1] + 1 #la variable siguiente a la guardada
#Si un modelo con la última variable ya fue entrenado
if index_init == 3:
  index_init = 0 #reinicia el indice de variables
  key = key + 1 #aumenta el indice del .csv a leer
print(init_values)
print(key,index_init)

[75, 0]
75 1


In [ ]:
for index in range(key,LIMIT):
  csv = FILES[index]
  if csv not in NULL_TOWNS:
    #LEER EL CSV
    df = get_csv_data(f"{dataset_path}/{csv}")
    date_time = df.pop('FECHA')
    #SELECCIONAR UNA VARIABLE PARA ENTRENAR LA RED
    for index_var in range(index_init,len(VARIABLES)):
      var = VARIABLES[index_var]
      print(f'Entrenando con Archivo {csv} variable {var} index {index}')
      #Grafica los datos
      #plot_cols = [var]
      #plot_features = df[plot_cols]
      #plot_features.index = date_time
      #_ = plot_features.plot()

      #Training y testing
      train = df[var][:len(df[var])-365] #Todos exceptuando el último año
      test = df[var][len(df[var])-365:] #El último año

      #Windowed training para la red recurrente
      train_set = windowed_dataset(train, window_size, batch_size, shuffle_buffer_size)
      #Obteniendo el modelo de la red
      model = Neural_network_model()
      #Entrenando la red
      history = model.fit(train_set,epochs=200)
      #Guardando el modelo
      name_model = get_file_name(csv)
      name_var = DICT_NAMES[var]
      model.save(f'{models_path}{name_model}-{name_var}.h5')
      #Guarda el objeto history
      save_history(csv, name_var, history.history)
      #Guardando los valores de proceso
      save_last_key([index,index_var])
      #Limpia la consola
      clear_output()
      
    #Reiniciando key de indice de variable
    index_init = 0
    #Guarda en log
    bitacora(csv)
  else:
    bitacora(f'{csv} CON NULOS')

In [ ]:
#print(history.history)
print(HISTORY)

{'14002-ACATLAN DE JUAREZ.csv': {'precip': {'loss': [9.412681579589844, 6.334582805633545, 5.313165664672852, 4.836631774902344, 4.476109504699707, 4.257781982421875, 4.065696716308594, 3.901013135910034, 3.7563772201538086, 3.628579616546631, 3.519181251525879, 3.426830768585205, 3.3484842777252197, 3.27925705909729, 3.2152817249298096, 3.1581645011901855, 3.104590654373169, 3.0535714626312256, 3.0028727054595947, 2.9532721042633057, 2.9008121490478516, 2.8467726707458496, 2.793360948562622, 2.7390966415405273, 2.685750722885132, 2.6355905532836914, 2.5919651985168457, 2.5574820041656494, 2.530329942703247, 2.5071940422058105, 2.486821413040161, 2.4703454971313477, 2.4565536975860596, 2.445281505584717, 2.435236692428589, 2.426337480545044, 2.4182615280151367, 2.410738706588745, 2.403637409210205, 2.397420644760132, 2.39164400100708, 2.386058807373047, 2.3807337284088135, 2.3756041526794434, 2.3706302642822266, 2.365891456604004, 2.3613951206207275, 2.3573949337005615, 2.3535702228546

# **Entrenamiento de municipios con nulos**


**Eliminacion de nulos** 

In [61]:
NULL_TOWNS = ['14026-COLOTLAN.csv', '14037-HUEJUCAR.csv', '15027-SANTA MARIA DE LOS ANGELES.csv', '14324-TOTATICHE.csv']
csv = NULL_TOWNS[1]
df = get_csv_data(f"{dataset_path}/{csv}")
date_time = df.pop('FECHA')
VARIABLES = ['PRECIP(mm)', 'TMAX(°C)', 'TMIN(°C)']
#var = VARIABLES[0]
train = df[var][:len(df[var])-365] #Todos exceptuando el último año

def drop_nulls(series):
  cleaned_series = []
  for element in series:
    if element != 'Nulo':
      #print(type(element))
      cleaned_series.append(float(element))
  
  return pd.Series(cleaned_series)
  
print(len(train))
print(train[0])
print(type(train[0]))
cleaned_train = drop_nulls(train)
print(len(cleaned_train))
print(cleaned_train[0])
print(type(cleaned_train[0]))

3653
Nulo
<class 'str'>
3590
0.0
<class 'numpy.float64'>


In [62]:
key = 0
index_init = 0
tf.random.set_seed(51)
np.random.seed(51)
shuffle_buffer_size = 1000
window_size = 64
batch_size = 365
print(NULL_TOWNS)
print(VARIABLES)

['14026-COLOTLAN.csv', '14037-HUEJUCAR.csv', '15027-SANTA MARIA DE LOS ANGELES.csv', '14324-TOTATICHE.csv']
['PRECIP(mm)', 'TMAX(°C)', 'TMIN(°C)']


In [63]:
for index in range(key,len(NULL_TOWNS)):
  csv = NULL_TOWNS[index]
  #LEER EL CSV
  df = get_csv_data(f"{dataset_path}/{csv}")
  date_time = df.pop('FECHA')
  #SELECCIONAR UNA VARIABLE PARA ENTRENAR LA RED
  for index_var in range(index_init,len(VARIABLES)):
    var = VARIABLES[index_var]
    print(f'Entrenando con Archivo {csv} variable {var} index {index}')
    #Grafica los datos
    #plot_cols = [var]
    #plot_features = df[plot_cols]
    #plot_features.index = date_time
    #_ = plot_features.plot()

    #Training y testing
    train = drop_nulls(df[var][:len(df[var])-365]) #Todos exceptuando el último año
    test = drop_nulls(df[var][len(df[var])-365:]) #El último año

    #Windowed training para la red recurrente
    train_set = windowed_dataset(train, window_size, batch_size, shuffle_buffer_size)
    #Obteniendo el modelo de la red
    model = Neural_network_model()
    #Entrenando la red
    history = model.fit(train_set,epochs=200)
    #Guardando el modelo
    name_model = get_file_name(csv)
    name_var = DICT_NAMES[var]
    model.save(f'{models_path}{name_model}-{name_var}.h5')
    #Guarda el objeto history
    save_history(csv, name_var, history.history)
    #Guardando los valores de proceso
    #save_last_key([index,index_var])
    #Limpia la consola
    clear_output()
    
  #Reiniciando key de indice de variable
  index_init = 0
  #Guarda en log
  bitacora(f'{csv} SIN NULOS')